In [1]:
#使用inspect_checkpoint来查看ckpt里的内容
from tensorflow.python.tools import inspect_checkpoint as chkp

chkp.print_tensors_in_checkpoint_file(file_name="./models/resnet50_vild/model.ckpt-180000", 
     tensor_name=None, # 如果为None,则默认为ckpt里的所有变量
     all_tensors=True, # bool 是否打印所有的tensor，这里打印出的是tensor的值，一般不推荐这里设置为False
     all_tensor_names=True) # bool 是否打印所有的tensor的name


tensor: fpn/l2/bias (float32) [256]
[-2.47692596e-02 -3.90116661e-03  1.21684028e-02  7.20547047e-03
  9.45634171e-02 -3.99536565e-02  5.19875959e-02 -8.11061710e-02
 -7.19396472e-02  1.75216179e-02 -1.65722772e-01  1.23834312e-01
 -5.37701659e-02  8.03819299e-02 -1.52981626e-02 -9.75081995e-02
 -7.34366802e-03  5.54464534e-02 -1.58375390e-02  3.58207785e-02
  5.04227355e-02  9.78472270e-03  8.30873847e-03  4.36827280e-02
  1.18819565e-01 -3.08101494e-02  2.39665783e-03 -5.68323843e-02
  3.96851115e-02  1.31594101e-02  4.96311896e-02  1.35356188e-01
  7.07092062e-02 -1.11553840e-01 -7.01859742e-02 -5.12524173e-02
  4.27638181e-02  1.09757178e-01  5.54374717e-02  3.38252857e-02
  5.29881977e-02 -9.33772400e-02 -8.84771645e-02  1.48380339e-01
  1.05455734e-01  1.93719063e-02  9.67393667e-02 -1.24281809e-01
  1.11693814e-01 -2.00459696e-02 -8.75199661e-02  6.14735894e-02
 -9.75367576e-02 -7.97979906e-02  2.70166341e-02  7.08989613e-03
  5.93852922e-02 -5.21422252e-02  4.35791351e-02  3.05

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = ""
os.environ['CUDA_VISIBLE_DEVICES'] = ""

import numpy as np
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()

from collections import OrderedDict

In [3]:
'''
x = tf.train.load_variable("./models/resnet50_vild/model.ckpt-180000",
                                   "frcnn_layer_0/fast_rcnn_head/class-predict/kernel")
'''
#print(x.shape)
#print(type(x))

y = np.load('./fake_model8/fake8.npy')
#y = y.astype('float32')
#print(y)

var_shape_dict = OrderedDict(tf.train.list_variables("./models/resnet50_vild/model.ckpt-180000"))
var_name_list = list(var_shape_dict.keys())
var_data_dict = OrderedDict()

for var_name in var_name_list:
    var_data = tf.train.load_variable("./models/resnet50_vild/model.ckpt-180000", var_name)
    var_data_dict.update({var_name: var_data})

#print(var_data_dict["frcnn_layer_0/fast_rcnn_head/class-predict/kernel"].dtype)
#print(y.dtype)

var_shape_dict["frcnn_layer_0/fast_rcnn_head/class-predict/kernel"] = list(y.shape)
var_data_dict["frcnn_layer_0/fast_rcnn_head/class-predict/kernel"] = y.astype("float32")

#print(y.dtype())

with tf.Session() as sess:
    var_list = var_data_dict.items()
    saver_var_list = []
    with tf.Session() as sess2:
        for var_name, var_data in var_list:
            var = tf.Variable(var_data, name=var_name)
            saver_var_list.append(var)
            
    var_list = saver_var_list
    # Construct the Saver
    saver = tf.train.Saver(var_list=var_list)
    # Necessary! Call the initializer at the beginning.
    sess.run(tf.global_variables_initializer())
    saver.save(sess, "./fake_model8/model.ckpt-180000")
    print("Checkpoint saving finished !\n")
    

2022-07-11 01:56:56.798359: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-07-11 01:56:56.798380: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: clarity2
2022-07-11 01:56:56.798385: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: clarity2
2022-07-11 01:56:56.798429: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.73.8
2022-07-11 01:56:56.798445: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 510.73.8
2022-07-11 01:56:56.798450: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 510.73.8
2022-07-11 01:56:56.798617: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in 

INFO:tensorflow:./fake_model8/model.ckpt-180000.index
INFO:tensorflow:0
INFO:tensorflow:./fake_model8/model.ckpt-180000.data-00000-of-00001
INFO:tensorflow:384700
INFO:tensorflow:./fake_model8/model.ckpt-180000.meta
INFO:tensorflow:770700
Checkpoint saving finished !



In [4]:
x = tf.train.load_variable("./models/resnet50_vild/model.ckpt-180000",
                                   "frcnn_layer_0/fast_rcnn_head/class-predict/kernel")
print(x.shape)
#print(type(x))
xx = tf.train.load_variable("./fake_model8/model.ckpt-180000",
                                   "frcnn_layer_0/fast_rcnn_head/class-predict/kernel")
#print(xx)
print(xx.shape)

(512, 1203)
(512, 8)
